In [ ]:
# default_exp core

# Dezero Core

> Variable and Function

In [ ]:
#export
from dezero.imports import *

In [ ]:
#hide
from nbdev.showdoc import *

# Variable

In [ ]:
#export
class Variable():
    def __init__(self, data):
        self.data = data

### Variables as boxes
~~~python
class Variable():
    def __init__(self, data):
        self.data = data
~~~
可以把变量想象成一个盒子，盒子里面放的是数据。

![Variables as boxes](images/1-1.png)

In [ ]:
x = Variable(np.array(1.0))
x.data

array(1.)

In [ ]:
x.data = np.array(2.0)
x.data

array(2.)

# Function

In [ ]:
#export
class Function():
    def __call__(self, input:Variable):
        x = input.data
        y = self.forward(x)
        output = Variable(y)
        return output
    
    def forward(self, x):
        raise NotImplementedError()
        

函数是用来定义一个变量与另一个变量之间的对应关系

$y=f(x)$

![Variables as boxes](images/1-3.png)

### Function to create a variable

例如，实现 $ f=x^2 $
~~~python
class Function():
    def __call__(self, input:Variable):
        x = input.data
        y = self.forward(x)
        output = Variable(y)
        return output
    
    def forward(self, x):
        raise NotImplementedError()
~~~

In [ ]:
class Square(Function):
    def forward(self, x):
        return x ** 2

In [ ]:
f = Square()
f(Variable(2)).data

4

### Connecting Functions

In [ ]:
class Exp(Function):
    def forward(self, x):
        return np.exp(x)

多个函数可以进行串联来实现复杂的操作，例如，实现 $f(x)=(e^{x^2})^2$

In [ ]:
A = Square()
B = Exp()
C = Square()

x = Variable(np.array(0.5))
a = A(x)
b = B(a)
y = C(b)
y.data

1.648721270700128

计算图

![](images/1-4.png)


### Numerical Differentiation

> 数值微分(numerical differentiation)根据函数在一些离散点的函数值，推算它在某点的导数或高阶导数的近似值的方法。通常用差商代替微商，或者用一个能够近似代替该函数的较简单的可微函数（如多项式或样条函数等）的相应导数作为能求导数的近似值。

> $$f'(x)=\lim_{h \to 0}\frac{f(x+h)-f(x)}{h}$$
> 割线斜率和切线斜率有些差异，差异大约和h成正比。若h近似于0，则割线斜率近似于切线斜率。

![](images/1-5.png)

另外一种二点估计法是用经过$(x-h,f(x-h))$和$(x+h,f(x+h))$二点的割线，其斜率为$\frac{f(x+h)-f(x-h)}{2h}$.

上述公式称为对称差分，其一次项误差相消，因此割线斜率和切线斜率的差和成正比。对于很小的h而言这个值比单边近似还要准确。特别的是公式虽计算x点的斜率，但不会用到函数在x点的数值。


![](images/1-6en.png)

从图中可以看出对称差分相比单边差分近似更准确一些，下面就实现一个简单的对称差分算法

In [ ]:
def numerical_diff(f, x, eps=1e-4):
    x0 = Variable(x.data-eps)
    x1 = Variable(x.data+eps)
    y0 = f(x0)
    y1 = f(x1)
    return (y1.data - y0.data)/(2*eps)

In [ ]:
f = Square()
x = Variable(np.array(2.0))
dy = numerical_diff(f, x)
dy

4.000000000004

真实的导数按照 $f'(x)=2x$ 计算值为4，对称差分算法计算会有一定的误差。

In [ ]:
def f(x):
    A = Square()
    B = Exp()
    C = Square()
    return C(B(A(x)))
x = Variable(0.5)
dy = numerical_diff(f, x)
dy

3.2974426293330694

通过这种方式可以自动求复合函数的导数，在大部分情况这种方式计算出的误差比较小，但是某些计算可能会包含更大的误差。在神经网络中，有上百万的参数需要计算导数使用数值微分的方式会导致计算量非常大。